In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/asba-nlim/airy_test_nlim.csv
/kaggle/input/asba-nlim/airy_test_nlim2.csv
/kaggle/input/asba-nlim/rest_test_nlim.csv
/kaggle/input/asba-nlim/car_train_nlim.csv
/kaggle/input/asba-nlim/rest_train_nlim.csv
/kaggle/input/asba-nlim/airy_train_nlim.csv
/kaggle/input/asba-nlim/car_test_nlim.csv
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/corpus_info.json
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/model/config.json
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/model/model.ckpt.meta
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/model/model.ckpt.index
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/model/model.ckpt.data-00000-of-00001
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/model/checkpoint
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/model/pytorch_model/config.json
/kaggle/input/finetune-model-airy-corpus/experimenttfrecords/model/pytorch_model/pytorch_model.bin
/kagg

In [2]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-02-10 00:55:24.778375: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 00:55:24.853046: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 00:55:24.970071: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-10 00:55:24.971011: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [3]:
!pip install sentencepiece
!pip install pytorch-transformers
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 1.2 MB/s eta 0:00:00


In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from pytorch_transformers import XLNetModel
from transformers import XLNetForSequenceClassification, XLNetTokenizer, get_linear_schedule_with_warmup, AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from random import random
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [6]:
def truncate_seq_pair(tokens_a, tokens_b, max_num_tokens):
    """Truncates a pair of sequences to a maximum sequence length. Lifted from Google's BERT repo."""
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_num_tokens:
            break

        trunc_tokens = tokens_a if len(tokens_a) > len(tokens_b) else tokens_b
        assert len(trunc_tokens) >= 1

        # We want to sometimes truncate from the front and sometimes from the
        # back to add more randomness and avoid biases.
        if random() < 0.5:
            del trunc_tokens[0]
        else:
            trunc_tokens.pop()

In [7]:
def get_tokenized_texts_from(tokenizer, sentences_a, sentences_b, max_len):
    tokenized_texts_a = [tokenizer.tokenize(sent) for sent in sentences_a]
    tokenized_texts_b = [tokenizer.tokenize(sent) for sent in sentences_b]
    tokenized_texts = []
    for i in range (len(tokenized_texts_a)):
        truncate_seq_pair(tokenized_texts_a[i], tokenized_texts_b[i], max_len-3)
        tokens = tokenized_texts_a[i] + ["<sep>"] + tokenized_texts_b[i] + ["<sep>"] + ["<cls>"]
        tokenized_texts.append(tokens)
    return tokenized_texts

In [8]:
df = pd.read_csv("/kaggle/input/asba-nlim/car_train_nlim.csv")
df_test = pd.read_csv("/kaggle/input/asba-nlim/car_test_nlim.csv")

In [9]:
df.shape

(1075, 3)

In [10]:
df.head()

,text_a,text_b,label
0,Saya memakai Honda Jazz GK5 tahun 2014 ( perta...,others,1
1,Avanza kenapa jadi boros bensin begini dah ah....,fuel,0
2,"saran ku dan pengalaman ku , mending beli mobi...",fuel,1
3,"saran ku dan pengalaman ku , mending beli mobi...",machine,1
4,Dari segi harga juga pajero lebih mahal 30 jut...,price,1


In [11]:
df_train ,df_val = train_test_split(df, random_state=2018, test_size=0.2)

print('Number of training sentences: {:,}\n'.format(df_train.shape[0]))
print('Number of testing sentences: {:,}\n'.format(df_val.shape[0]))

Number of training sentences: 860

Number of testing sentences: 215



In [12]:
MAX_LEN = 128



In [13]:
sentences_a_train = df_train.text_a.values
sentences_b_train = df_train.text_b.values
labels_train = df_train.label.values

In [14]:
sentences_a_val = df_val.text_a.values
sentences_b_val = df_val.text_b.values
labels_val = df_val.label.values

In [15]:
sentences_a_test = df_test.text_a.values
sentences_b_test = df_test.text_b.values
labels_test = df_test.label.values

In [16]:
# sentences[0]

In [17]:
tokenizer = XLNetTokenizer.from_pretrained('malay-huggingface/xlnet-base-bahasa-cased', do_lower_case=True)
tokenized_texts_train = get_tokenized_texts_from(tokenizer, sentences_a_train, sentences_b_train, MAX_LEN)
tokenized_texts_val = get_tokenized_texts_from(tokenizer, sentences_a_val, sentences_b_val, MAX_LEN)
tokenized_texts_test = get_tokenized_texts_from(tokenizer, sentences_a_test, sentences_b_test, MAX_LEN)

print ("Tokenize the first sentence:")
print (tokenized_texts_train[0])

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/290 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Tokenize the first sentence:
['▁mobil', '▁ku', '▁', 'dai', 'hat', 'su', '▁x', 'e', 'nia', '▁ku', '▁i', 'rit', '▁,', '▁cocok', '▁banget', '▁buat', '▁keluarga', '▁indonesia', '▁dengan', '▁kursi', '▁7', '▁tambah', '▁lagi', '▁mesin', '▁handal', '▁', '(', '▁1', '.', '300', '▁cc', '▁', ')', '▁walau', '▁sudah', '▁sejak', '▁2009', '<sep>', '▁part', '<sep>', '<cls>']


In [18]:
input_ids_train = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]
input_ids_val = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_val]
input_ids_test = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]

In [19]:
input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids_val = pad_sequences(input_ids_val, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


In [20]:
tokenized_texts_test =pad_sequences(tokenized_texts_test, maxlen=MAX_LEN, dtype=object, truncating="post", padding="post")

df_test['tokens'] = tokenized_texts_test.tolist()

In [21]:
# Create attention masks
attention_masks_train = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids_train:
  seq_mask_train = [float(i>0) for i in seq]
  attention_masks_train.append(seq_mask_train)

# Create attention masks
attention_masks_val = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids_val:
  seq_mask_val = [float(i>0) for i in seq]
  attention_masks_val.append(seq_mask_val)
    
attention_masks_test = []
for seq in input_ids_test:
  seq_mask_test = [float(i>0) for i in seq]
  attention_masks_test.append(seq_mask_test)

In [22]:
train_inputs = torch.tensor(input_ids_train)
train_labels = torch.tensor(labels_train)
train_masks = torch.tensor(attention_masks_train)

val_inputs = torch.tensor(input_ids_val)
val_labels = torch.tensor(labels_val)
val_masks = torch.tensor(attention_masks_val)

test_inputs = torch.tensor(input_ids_test)
test_labels = torch.tensor(labels_test)
test_masks = torch.tensor(attention_masks_test)

In [23]:
batch_size = 32
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

Using **finetuned language model**

In [24]:
# Load XLNEtForSequenceClassification, the pretrained XLNet model with a single linear classification layer on top. 
model = XLNetForSequenceClassification.from_pretrained('/kaggle/input/finetune-model-any/r_experimenttfrecords/model/r_pytorch_model', num_labels=2)
model.cuda()

Some weights of the model checkpoint at /kaggle/input/finetune-model-any/r_experimenttfrecords/model/r_pytorch_model were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/finetune-model-any/r_experimenttfrecords/model/r_pytorch_model and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [25]:
epochs = 8

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [26]:
# Store our loss and accuracy for plotting
def train_epoch(model, data_loader, optimizer, device, scheduler):
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]   
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))


    

In [27]:
def eval_model(model, data_loader, device):
    model = model.eval()
    correct = 0
    total = 0
    predictions =[]
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
          batch = tuple(t.to(device) for t in batch)
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          # print (outputs)
          prediction = torch.argmax(outputs[0],dim=1)
          predictions = predictions + prediction.tolist()
          total += b_labels.size(0)
          correct+=(prediction==b_labels).sum().item()
    return correct / total, f1_score(labels_val, predictions, average='macro')

In [28]:
best_accuracy = 0
best_f1 = 0

# trange is a tqdm wrapper around the normal python range
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')
    print('-' * 10)
    
    train_epoch(
        model,
        train_dataloader,     
        optimizer, 
        device, 
        scheduler
    )
    
    val_acc, val_f1 = eval_model(
        model,
        val_dataloader, 
        device
    )
    
    print(f' Val accuracy {val_acc}')
    print(f' Val f1 {val_f1}')
    print()
    if val_f1 > best_f1:
        torch.save(model.state_dict(), '/kaggle/working/best_car_post_rest.ckpt')
        best_f1 = val_f1

print(f' F1 Validation {val_f1}')
  
torch.save(model.state_dict(), '/kaggle/working/last_car_post-rest.ckpt')
  

Epoch 1/8
----------
Train loss: 0.5757881005605062
 Val accuracy 0.7023255813953488
 Val f1 0.412568306010929

Epoch 2/8
----------
Train loss: 0.48969810649200723
 Val accuracy 0.8093023255813954
 Val f1 0.7230164964650432

Epoch 3/8
----------
Train loss: 0.34883716647271756
 Val accuracy 0.9209302325581395
 Val f1 0.9032070125261513

Epoch 4/8
----------
Train loss: 0.2351074845150665
 Val accuracy 0.9162790697674419
 Val f1 0.8959677419354839

Epoch 5/8
----------
Train loss: 0.14176553232526337
 Val accuracy 0.9209302325581395
 Val f1 0.9050180608612042

Epoch 6/8
----------
Train loss: 0.13554873276087973
 Val accuracy 0.9209302325581395
 Val f1 0.9022439755008158

Epoch 7/8
----------
Train loss: 0.12423149951630169
 Val accuracy 0.9209302325581395
 Val f1 0.9032070125261513

Epoch 8/8
----------
Train loss: 0.08116781359745397
 Val accuracy 0.9255813953488372
 Val f1 0.9084522035341708

 F1 Validation 0.9084522035341708
